In [8]:
# 本代码由可视化策略环境自动生成 2021年12月29日 10:55
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m5_initialize_bigquant_run(context):
    # 加载股票指标数据,数据继承自m6模块
    context.indicator_data = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    
    # 设置股票数量
    context.stock_num = 30
    
    # 调仓天数,22个交易日大概就是一个月。可以理解为一个月换仓一次
    context.rebalance_days = 22
    
    # 如果策略运行中，需要将数据进行保存，可以借用extension这个对象，类型为dict
    # 比如当前运行的k线的索引，比如个股持仓天数、买入均价
    if 'index' not in context.extension:
        context.extension['index'] = 0
 
    

# 回测引擎：每日数据处理函数，每天执行一次
def m5_handle_data_bigquant_run(context, data):
    
    
    context.extension['index'] += 1
    # 不在换仓日就return,相当于后面的代码只会一个月运行一次，买入的股票会持有1个月
    if  context.extension['index'] % context.rebalance_days != 0:
        return 
    
    # 当前的日期
    date = data.current_dt.strftime('%Y-%m-%d')
    
    cur_data = context.indicator_data[context.indicator_data['date'] == date]
    # 根据日期获取调仓需要买入的股票的列表
    stock_to_buy = list(cur_data.instrument[:context.stock_num])
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式

        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            context.order_target_percent(context.symbol(stock), 0)
    
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return

    # 等权重买入 
    weight =  1 / len(stock_to_buy)
    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            # 下单使得某只股票的持仓权重达到weight，因为
            # weight大于0,因此是等权重买入
            context.order_target_percent(context.symbol(stock), weight)
 
# 回测引擎：准备数据，只执行一次
def m5_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m5_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.use_datasource.v1(
    datasource_id='bar1d_CN_FUND',
    start_date='2020-01-01',
    end_date='2021-01-01'
)

m2 = M.use_datasource.v1(
    datasource_id='history_nav_CN_FUND',
    start_date='2020-01-01',
    end_date='2021-01-01'
)

m7 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
nav_chg_pct"""
)

m8 = M.derived_feature_extractor.v3(
    input_data=m2.data,
    features=m7.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m3 = M.join.v3(
    data1=m1.data,
    data2=m8.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m4 = M.sort.v4(
    input_ds=m3.data,
    sort_by='nav_chg_pct',
    group_by='instrument',
    keep_columns='--',
    ascending=True
)

m6 = M.filter.v3(
    input_data=m4.sorted_data,
    expr='amount > 3000',
    output_left_data=False
)

m5 = M.trade.v4(
    options_data=m6.data,
    start_date='2020-01-01',
    end_date='2021-01-01',
    initialize=m5_initialize_bigquant_run,
    handle_data=m5_handle_data_bigquant_run,
    prepare=m5_prepare_bigquant_run,
    before_trading_start=m5_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-12-29 10:54:01.510355] INFO: moduleinvoker: use_datasource.v1 开始运行..

[2021-12-29 10:54:01.520804] INFO: moduleinvoker: 命中缓存

[2021-12-29 10:54:01.523213] INFO: moduleinvoker: use_datasource.v1 运行完成[0.012864s].

[2021-12-29 10:54:01.530601] INFO: moduleinvoker: use_datasource.v1 开始运行..

[2021-12-29 10:54:01.538574] INFO: moduleinvoker: 命中缓存

...

[2021-12-29 10:54:02.525608] INFO: moduleinvoker: 命中缓存

[2021-12-29 10:54:02.527858] INFO: moduleinvoker: sort.v4 运行完成[0.011329s].

[2021-12-29 10:54:02.537687] INFO: moduleinvoker: filter.v3 开始运行..

[2021-12-29 10:54:02.547344] INFO: moduleinvoker: 命中缓存

[2021-12-29 10:54:02.549374] INFO: moduleinvoker: filter.v3 运行完成[0.01187s].

In [9]:
m6.data.read()

,instrument,date,end_date,nav,cumulative_nav,adjust_nav,publish_date,nav_chg,nav_chg_pct,nav_chg_pct_adj,open,high,low,close,volume,amount,adjust_factor
0,150221.ZOF,2020-01-02,2020-01-02,1.0000,1.3170,1.3530,2020-01-02,-0.0650,-6.1033,-0.0074,1.8147,1.8198,1.8130,1.8164,6494782.0,6.982747e+06,0.591830
1,150219.ZOF,2020-01-02,2020-01-02,1.0000,1.2900,1.3211,2020-01-02,-0.0600,-5.6604,0.0000,1.9950,1.9950,1.9950,1.9950,39000.0,4.520100e+04,0.580952
2,150123.ZOF,2020-01-02,2020-01-02,1.0003,1.4451,1.5027,2020-01-02,-0.0597,-5.6321,0.0332,2.2733,2.2754,2.2733,2.2733,407800.0,4.330704e+05,0.466722
3,150130.ZOF,2020-01-02,2020-01-02,1.0000,1.3803,1.4571,2020-01-02,-0.0567,-5.3658,0.0274,1.3903,1.3969,1.3836,1.3969,499177.0,5.259953e+05,0.756675
4,150190.ZOF,2020-01-02,2020-01-02,1.0000,1.3490,1.3438,2020-01-02,-0.0550,-5.2133,0.0000,1.2989,1.3725,1.2989,1.3663,26907.0,2.920381e+04,0.801435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205376,150236.ZOF,2020-12-31,2020-12-31,1.4260,0.1120,0.1298,2020-12-31,0.0990,7.4604,7.4503,0.2241,0.2438,0.2241,0.2408,2297568.0,3.152809e+06,5.739202
205377,150302.ZOF,2020-12-31,2020-12-31,1.4216,0.1227,0.1227,2020-12-31,0.0991,7.4934,7.5372,0.2268,0.2386,0.2268,0.2360,691223.0,9.470867e+05,5.838983
205378,150172.ZOF,2020-12-31,2020-12-31,1.0666,2.8419,0.2492,2020-12-31,0.0776,7.8463,7.8321,1.0804,1.1746,1.0804,1.1634,108671931.0,1.111816e+08,0.892212
205379,150224.ZOF,2020-12-31,2020-12-31,1.2690,0.0490,0.0552,2020-12-31,0.0950,8.0920,8.0234,0.2339,0.2526,0.2339,0.2506,126755702.0,1.542177e+08,4.920191
